In [1]:
from json import loads
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings("ignore")
from pyspark.sql.functions import col,from_json,udf,split,explode
from pyspark.ml.feature import NGram
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,MapType,FloatType,ArrayType

In [2]:
spark = SparkSession.\
        builder.\
        appName("comment-anlysis").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1024m").\
        getOrCreate()

23/01/25 12:57:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df_product = spark.read.parquet('hdfs://namenode:9000/TikiCleaned/Product')
df_comment = spark.read.parquet('hdfs://namenode:9000/TikiCleaned/Comment')
df_shop = spark.read.parquet('hdfs://namenode:9000/TikiCleaned/ShopInfo')

In [4]:
df_product.count()

95464

In [5]:
df_comment.count()

3568061

In [6]:
df_shop.count()

8686

In [4]:
df_shop.printSchema()

root
 |-- id: integer (nullable = true)
 |-- store_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- icon: string (nullable = true)
 |-- url: string (nullable = true)
 |-- is_official: boolean (nullable = true)
 |-- store_level: string (nullable = true)
 |-- is_followed: boolean (nullable = true)
 |-- avg_rating_point: float (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- total_follower: integer (nullable = true)
 |-- days_since_joined: integer (nullable = true)



In [5]:
df_comment.printSchema()

root
 |-- id: integer (nullable = true)
 |-- content: string (nullable = true)
 |-- clean_content: string (nullable = true)
 |-- thank_count: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- spid: integer (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- customer_full_name: string (nullable = true)
 |-- purchased_at: string (nullable = true)
 |-- date_purchased_at: string (nullable = true)
 |-- review_created_date: string (nullable = true)
 |-- delivery_date: string (nullable = true)
 |-- review_after_delivery: integer (nullable = true)
 |-- sentiment: string (nullable = true)
 |-- lable: integer (nullable = true)



In [8]:
df_product.printSchema()

root
 |-- id: integer (nullable = true)
 |-- master_id: integer (nullable = true)
 |-- sku: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- list_price: integer (nullable = true)
 |-- original_price: integer (nullable = true)
 |-- discount: integer (nullable = true)
 |-- discount_rate: float (nullable = true)
 |-- rating_average: float (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- productset_group_name: string (nullable = true)
 |-- all_time_quantity_sold: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- clean_name: string (nullable = true)
 |-- clean_description: string (nullable = true)
 |-- clean_specifications: string (nullable = true)
 |-- category_name: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- seller_store_id: string (nullable = true)
 |-- spid: integer (nullable = true)
 |-- category_id: integer (null

In [7]:
Product_Shop = spark.read.parquet('hdfs://namenode:9000/analysis/Product_Shop')
metaData = spark.read.parquet('hdfs://namenode:9000/analysis/metaData')
comment_1gram = spark.read.parquet('hdfs://namenode:9000/analysis/comment_1gram')
comment_2gram = spark.read.parquet('hdfs://namenode:9000/analysis/comment_2gram')

In [12]:
Product_Shop.count()

23/01/25 13:31:19 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


95451

In [13]:
metaData.count()

3038114

In [14]:
comment_1gram.count()

81038

In [15]:
comment_2gram.count()

1863421

In [10]:
comment_1gram.printSchema()

root
 |-- sentiment: string (nullable = true)
 |-- word: string (nullable = true)
 |-- count: long (nullable = true)



In [11]:
comment_2gram.printSchema()

root
 |-- sentiment: string (nullable = true)
 |-- word: string (nullable = true)
 |-- count: long (nullable = true)



In [8]:
Product_Shop.printSchema()

root
 |-- p_id: integer (nullable = true)
 |-- master_id: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- list_price: integer (nullable = true)
 |-- original_price: integer (nullable = true)
 |-- discount: integer (nullable = true)
 |-- discount_rate: float (nullable = true)
 |-- productset_group_name: string (nullable = true)
 |-- all_time_quantity_sold: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- p_clean_name: string (nullable = true)
 |-- clean_description: string (nullable = true)
 |-- clean_specifications: string (nullable = true)
 |-- category_name: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- store_id: integer (nullable = true)
 |-- s_name: string (nullable = true)
 |-- icon: string (nullable = true)
 |-- url: string (nullable = true)
 |-- is_official: boolean (nullable = true)
 |-- store_level: string (nullable = true)
 |-- is_followed: boolean (nullable = tr

In [9]:
metaData.printSchema()

root
 |-- p_id: integer (nullable = true)
 |-- master_id: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- list_price: integer (nullable = true)
 |-- original_price: integer (nullable = true)
 |-- discount: integer (nullable = true)
 |-- discount_rate: float (nullable = true)
 |-- productset_group_name: string (nullable = true)
 |-- all_time_quantity_sold: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- p_clean_name: string (nullable = true)
 |-- clean_description: string (nullable = true)
 |-- clean_specifications: string (nullable = true)
 |-- category_name: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- store_id: integer (nullable = true)
 |-- s_name: string (nullable = true)
 |-- icon: string (nullable = true)
 |-- url: string (nullable = true)
 |-- is_official: boolean (nullable = true)
 |-- store_level: string (nullable = true)
 |-- is_followed: boolean (nullable = tr

In [3]:
df_product = spark.read.parquet('hdfs://namenode:9000/tiki/Product')
df_comment = spark.read.parquet('hdfs://namenode:9000/tiki/Comment')
df_shop = spark.read.parquet('hdfs://namenode:9000/tiki/ShopInfo')

In [22]:
df_product.show(10,120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                   value|
+------------------------------------------------------------------------------------------------------------------------+
|{"id": 24629723, "master_id": 24629723, "sku": "3201131389423", "name": "K\u00ednh M\u00e1t Nam Cao C\u1ea5p K\u00e8m...|
|{"id": 614854, "master_id": 614854, "sku": "2510608412924", "name": "Nh\u1eefng Gi\u1ea5c M\u01a1 \u1ede Hi\u1ec7u S\...|
|{"id": 62231649, "master_id": 62231649, "sku": "6840385438104", "name": "Combo 3 c\u1eb7p thanh ch\u1eafn khe c\u1eed...|
|{"id": 151756158, "master_id": 151756158, "sku": "1823652985285", "name": "N\u1ed2I \u00c1P SU\u1ea4T BLUESTONE PCB-5...|
|{"id": 614854, "master_id": 614854, "sku": "2510608412924", "name": "Nh\u1eefng Gi\u1ea5c M\u01a1 \u1ede Hi\u1ec7u S\...|
|{"id": 24629723

In [23]:
df_comment.show(10,120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                   value|
+------------------------------------------------------------------------------------------------------------------------+
|{"id": 12696578, "title": "B\u00ecnh th\u01b0\u1eddng", "content": "S\u1ea3n ph\u1ea9m Tiki Trading th\u00ec okie. M\...|
|{"id": 17837163, "title": "C\u1ef1c k\u00ec h\u00e0i l\u00f2ng", "content": "Pepsi chai nh\u1ecf. U\u1ed1ng ok nha. T...|
|{"id": 17656856, "title": "C\u1ef1c k\u00ec h\u00e0i l\u00f2ng", "content": "Giao h\u00e0ng nhanh, date m\u1edbi, th\...|
|{"id": 16628410, "title": "C\u1ef1c k\u00ec h\u00e0i l\u00f2ng", "content": "d\u00ed\u00edidididididiidid\u00ecvivivi...|
|{"id": 10268605, "title": "C\u1ef1c k\u00ec h\u00e0i l\u00f2ng", "content": "h\u00e0ng ngon ae nh\u00e9, n\u00ean mua...|
|{"id": 16265495

In [24]:
df_shop.show(10,120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                   value|
+------------------------------------------------------------------------------------------------------------------------+
|{"id": 13239, "store_id": 12900, "name": "Jess shop cosmetics", "icon": "https://vcdn.tikicdn.com/ts/seller/d5/cd/ca/...|
|{"id": 232650, "store_id": 219682, "name": "Hasu Sago Shop", "icon": "https://vcdn.tikicdn.com/ts/seller/11/87/fe/d55...|
|{"id": 31606, "store_id": 30924, "name": "H89 store", "icon": "https://vcdn.tikicdn.com/ts/seller/86/23/f2/5a4de26395...|
|{"id": 251877, "store_id": 238909, "name": "Gi\u00e0y d\u00e9p RYNA", "icon": "https://vcdn.tikicdn.com/ts/seller/95/...|
|{"id": 230813, "store_id": 217845, "name": "ROC LTD", "icon": "https://vcdn.tikicdn.com/ts/seller/c8/34/61/27f356bfc2...|
|{"id": 276580, 

In [10]:
from pyspark.sql.functions import sum

In [16]:
df_product.select(sum('all_time_quantity_sold')).collect()

[Row(sum(all_time_quantity_sold)=25207914)]

In [17]:
df_product_shop.select(sum('all_time_quantity_sold')).collect()

[Row(sum(all_time_quantity_sold)=25126552)]

In [13]:
df_product.select('all_time_quantity_sold').take(10)

[Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=8),
 Row(all_time_quantity_sold=1),
 Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=None)]

In [12]:
df_product_shop.select('all_time_quantity_sold').take(10)

[Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=8),
 Row(all_time_quantity_sold=1),
 Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=None)]

In [ ]:
df_product_shop.all_time_quantity_sold.sum()